In [1]:
from itertools import product
import altair as alt
import numpy as np
import pandas as pd

import gif

In [2]:
np.random.seed(2)

In [3]:
df = pd.DataFrame({
    'time': [1] * 3 + [2] * 3 + [3] * 3 + [4] * 3 + [5] * 3,
    'category': ['🥫', '🥜', '🕰'] * 5,
    'count': np.random.randint(low=1, high=10, size=3*5)
})

In [4]:
df['emoji'] = df.apply(lambda row: [row.category] * row['count'], axis=1)
df = df.explode('emoji')
df['column'] = df.groupby(['time', 'category']).rank(method='first')
df['column'] = df['column'].apply(int)
idx = list(product(
    range(1, df['time'].max() + 1),
    df['category'].unique(), 
    range(1, df['column'].max() + 1)
))
df = df.set_index(['time', 'category', 'column']).reindex(idx).reset_index()
df['emoji'] = df['emoji'].fillna('')

In [5]:
@gif.frame
def plot(t):
    d = df[df['time'] == t]

    chart = alt.Chart(d).mark_text().encode(
        alt.X('column:O', axis=None),
        alt.Y('category:O', axis=None),
        alt.SizeValue(15),
        text='emoji'
    ).properties(width=300, height=200)
    
    return chart

In [6]:
frames = []
for t in range(1, 5+1):
    frame = plot(t)
    frames.append(frame)

In [7]:
gif.save(frames, 'emoji.gif', duration=300)